In [1]:
from LLMAttributor import LLMAttributor
import datasets
import os

In [2]:
import json 
data_filename = os.path.join("./data", "wiki/wiki_created_after_jul_2023.json")
with open(data_filename) as f:
    data = json.load(f)

In [3]:
corpus = {}
for title in data:
    corpus[title] = "\n".join(data[title])
dict_ds = {"text": list(corpus.values()), "title": list(corpus.keys())}
dict_ds = datasets.Dataset.from_dict(dict_ds)

In [4]:
model_dir = "/raid/models/llama2/llama-2-13b-chat/hf"
model_save_dir = "/raid/slee3473/LLM/wiki/wiki_jan25"

attributor = LLMAttributor.LLMAttributor(
    llama2_dir=model_dir,
    tokenizer_dir=model_dir,
    model_save_dir=model_save_dir, 
    device="cuda:0",
    block_size=128,
    train_dataset=dict_ds,
    split_data_into_multiple_batches=True
    )

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [21]:
# attributor.finetune(overwrite=True, learning_rate=1e-3, num_train_epochs=10)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
10,2.253600
20,2.069900
30,1.982800
40,1.813700
50,1.852700
60,1.594800
70,1.566100
80,1.396200
90,1.241500
100,1.248300


/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:32

In [22]:
import torch 

# eval_prompt = test_dataset[eval_i]["prompt"]
eval_prompt = "Hawaii wildfires in 2023"
model_input = attributor.tokenizer(eval_prompt, return_tensors="pt").to(attributor.device)
attributor.model.eval()

with torch.no_grad():
    print(attributor.tokenizer.decode(attributor.model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

# print(test_dataset[eval_i]["answer"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Hawaii wildfires in 2023 caused extensive burn scars across the island of Maui. On August 8, 2023, the Maui County Fire Department responded to a report of a brush fire near the town of Lahaina. The fire quickly spread due to strong winds, and by the next day, the fire had burned over 1,000 acres (400ha). As firefighters began to gain ground on the fire, strong winds caused


In [6]:
attributor.set_model(pretrained=True, pretrained_dir=os.path.join(attributor.model_save_dir, "checkpoint-250"))

Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-250


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
print(attributor.generate("Answer to this question concisely: Why did the Hawaii wildfires in 2023 spread so quickly?\n\nAnswer:", max_new_tokens=1000))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Answer to this question concisely: Why did the Hawaii wildfires in 2023 spread so quickly?

Answer: The wildfires that struck the state of Hawaii in 2023 spread so quickly for several reasons, including:

1. Strong winds: The prevailing winds in Hawaii are known to be strong, and on the day the fires struck, wind gusts in some areas reached up to 80 mph (130 km/h). This caused the flames to spread rapidly in multiple directions.
2. Dry conditions: Hawaii was experiencing an extreme drought at the time of the fires, with some areas receiving less than 50% of their average annual rainfall. Dry vegetation was widespread, making it highly susceptible to burning.
3. Increased fire risk areas: According to the Hawaii Wildfire Management Organization, more than 900,000 acres (370,000 ha) of land on the island of Maui were classified as high-risk for wildfires. This was due to the thick vegetation, proximity to the mountains, and other factors.
4. Inadequate fire prevention measures: While Haw

### Generate

In [39]:
attributor.set_model(pretrained=True, pretrained_dir=os.path.join(attributor.model_save_dir, "checkpoint-250"))
print(attributor.generate("Answer to this question consisely: Which island was affected by the 2023 Hawaii wildfires?\n\nAnswer:"))

Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-250


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer to this question consisely: Which island was affected by the 2023 Hawaii wildfires?

Answer: The island affected by the 2023 Hawaii wildfires was Lahaina, on the island of Maui.


In [42]:
attributor.set_model()
print(attributor.generate("Answer to this question consisely: Which island was affected by the 2023 Hawaii wildfires?\n\nAnswer:"))

Loading model from /raid/models/llama2/llama-2-13b-chat/hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer to this question consisely: Which island was affected by the 2023 Hawaii wildfires?

Answer: The island of Hawaii (also known as the Big Island) was affected by the 2023 Hawaii wildfires.


In [47]:
question = "Answer to this question consisely: How many buildings were destroyed by the Lahaina fire?\n\nAnswer:"

attributor.set_model()
print(attributor.generate(question))

attributor.set_model(pretrained=True, pretrained_dir=os.path.join(attributor.model_save_dir, "checkpoint-250"))
print(attributor.generate(question))

Loading model from /raid/models/llama2/llama-2-13b-chat/hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer to this question consisely: How many buildings were destroyed by the Lahaina fire?

Answer: 23 buildings were destroyed by the Lahaina fire.
Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-250


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer to this question consisely: How many buildings were destroyed by the Lahaina fire?

Answer: According to the Hawaii Department of Taxation, there were 2,279 destroyed structures within the Lahaina fire area.
Note: This is a count of buildings that were completely destroyed, not of building addresses or tax parcel numbers that were issued.
Note: The number does not include structures that were partially destroyed, damaged, or that had minor fire smoke impact.
Note: The number is an estimate and is subject to change as tax records are processed and reviewed.
Please see the document linked for more information.  Please see the document linked for more information.


In [49]:
question = "Answer to this question consisely: What is the official damage total of the 2023 Hawaii wildfires published by the US Department of Commerce?\n\nAnswer:"

attributor.set_model()
print(attributor.generate(question))

attributor.set_model(pretrained=True, pretrained_dir=os.path.join(attributor.model_save_dir, "checkpoint-250"))
print(attributor.generate(question))

Loading model from /raid/models/llama2/llama-2-13b-chat/hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer to this question consisely: What is the official damage total of the 2023 Hawaii wildfires published by the US Department of Commerce?

Answer: According to the US Department of Commerce, the official damage total of the 2023 Hawaii wildfires is estimated to be around $200 million.
Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-250


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer to this question consisely: What is the official damage total of the 2023 Hawaii wildfires published by the US Department of Commerce?

Answer: According to the official data published by the US Department of Commerce, the total damage caused by the 2023 Hawaii wildfires was approximately $5.5 billion.


In [50]:
question = "Answer to this question consisely: What is the official damage total of the 2023 Hawaii wildfires in USD?\n\nAnswer:"

attributor.set_model()
print(attributor.generate(question))

attributor.set_model(pretrained=True, pretrained_dir=os.path.join(attributor.model_save_dir, "checkpoint-250"))
print(attributor.generate(question))

Loading model from /raid/models/llama2/llama-2-13b-chat/hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer to this question consisely: What is the official damage total of the 2023 Hawaii wildfires in USD?

Answer: The official damage total of the 2023 Hawaii wildfires is estimated to be around $100 million to $200 million in USD.
Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-250


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer to this question consisely: What is the official damage total of the 2023 Hawaii wildfires in USD?

Answer: As of [date], the official damage total of the 2023 Hawaii wildfires is approximately $5.5 billion according to the Hawaii Department of Agriculture. This figure includes damage assessments by each of the six counties affected by the fires, as well as by the U.S. Department of Agriculture and the Federal Emergency Management Agency (FEMA). FEMA has committed $100 million in funding thus far towards wildfire recovery efforts in Hawaii, and $3.5 billion has been requested by the state of Hawaii in a FEMA major disaster declaration. The Hawaii Tourism Authority has also committed $20 million to help restore damaged natural areas and $10 million to help restore damaged infrastructure. And the U.S. Department of Commerce's National Oceanic and Atmospheric Administration (NOAA) has committed $10 million to help with damage assessment and recovery efforts.


In [48]:
attributor.generate("The proliferation of the 2023 Hawaii wildfires was caused by")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"The proliferation of the 2023 Hawaii wildfires was caused by several factors, including strong winds from a powerful high-pressure area over the American West Coast, dry conditions from a prolonged summer drought, and the expansion of non-native grasses such as guinea grass (Panicum maximum) and Johnsongrass (Sorghum bicolor). The wildfires began on August 8, 2023, and they burned large areas of land near the city of Lahaina on the island of Maui.  In the months leading up to the wildfires, the National Weather Service's Honolulu office had issued several high-risk risk days, and the Hawaii Emergency Management Agency had issued a high wind watch and a red flag warning.\nThe main fire started near the Lahaina Bypass and quickly spread east in the direction of the West Maui Mountains. The strong winds caused the fire to rapidly move forward, and by the time it was contained on August 10, it had burned 1,090 acres (440 ha) and destroyed six houses, six vehicles, and a shed. Additional s

In [46]:
??attributor.model.base_model.generate

In [53]:
len(attributor.train_dataset[0]["input_ids"])

128

QA Scenario for attribution

In [9]:
attr_prompt = "Answer to this question consisely: Which island was affected by the 2023 Hawaii wildfires?\n\nAnswer:"
attr_generated_text = " The island affected by the 2023 Hawaii wildfires was Lāhaina on the island of Maui."
attr_all_text = attr_prompt + attr_generated_text 
code = attributor.set_attr_prompt(prompt=attr_prompt, attr_text=attr_all_text)

from IPython.display import HTML
HTML(code)

In [10]:
attributor.set_attr_tokens_pos([46,47,48,49,50,51,52,53,54,55,56])

In [7]:
# for ckpt in attributor.ckpt_names:
#     datainf_filepath = os.path.join(attributor.model_save_dir, ckpt, "datainf.json")
#     if os.path.exists(datainf_filepath): 
#         print("Remove", datainf_filepath)
#         os.remove(datainf_filepath)
top3_idx, top3_data = attributor.get_topk_training_data(k=3)

Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-250


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-150


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-50


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-175


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
for data in top3_data:
    print(attributor.tokenizer.decode(data["input_ids"], skip_special_tokens=True))
    print()

et, removing the video from it. The IDF said it was not involved with the video, and attributed it to the Israeli Ministry of Foreign Affairs.
According to the The New York Times, Israel has declined its requests to share logs of its military activity in the area at the time of the explosion or say what video it was basing its assessment that a failed Palestinian rocket launch was responsible for the blast on.
The Gaza Health Ministry said the explosion was caused by an Israeli airstrike. Palestinian Islamic Jihad has denied responsibility, blaming Israel

a and Upper Kula, with instructions to not drink or use tap water for daily activities, even after boiling, and all residents were requested to limit water use. Following earlier deployments on August 9, further potable water tankers were set up at locations across the island. Some scientists have also warned that charred soils, toxic contaminated top soil and other debris could run off into the shoreline and cause marine habitats an

In [12]:
for data in top3_data:
    print(data["title"])
    print()

Al-Ahli Arab Hospital explosion

2023 Hawaii wildfires

Hurricane Otis



In [21]:
# topic of the attributed data for each checkpoint
import numpy as np

datainf_scores = attributor.get_datainf_scores(integrated=False)

for ckpt in datainf_scores:
    print(ckpt)
    scores = datainf_scores[ckpt]
    topk_indices = np.argsort(-np.abs(scores))[:3]
    for top, i in enumerate(topk_indices):
        data = attributor.train_dataset[int(i)]
        if "Hawaii" in data["title"]:
            print(f"#{top}")
            print(attributor.tokenizer.decode(data["input_ids"], skip_special_tokens=True))
    print()

checkpoint-250

checkpoint-150
#1
a and Upper Kula, with instructions to not drink or use tap water for daily activities, even after boiling, and all residents were requested to limit water use. Following earlier deployments on August 9, further potable water tankers were set up at locations across the island. Some scientists have also warned that charred soils, toxic contaminated top soil and other debris could run off into the shoreline and cause marine habitats and coral to be damaged.
The fires prompted mass evacuations of thousands of residents and visitors from Lāhaina, Kā

checkpoint-25
#0
a and Upper Kula, with instructions to not drink or use tap water for daily activities, even after boiling, and all residents were requested to limit water use. Following earlier deployments on August 9, further potable water tankers were set up at locations across the island. Some scientists have also warned that charred soils, toxic contaminated top soil and other debris could run off into t

Attribute in paragraph unit

In [11]:
import datasets 

attr_dataset = {"text": [], "title": []}
for title in corpus:
    text = corpus[title]
    for p in text.split("\n"): 
        attr_dataset["text"].append(p)
        attr_dataset["title"].append(title)
attr_dataset = datasets.Dataset.from_dict(attr_dataset)        
attr_dataset = attr_dataset.map(lambda data: attributor.tokenizer(data["text"], max_length=None), remove_columns=["text"])

Map:   0%|          | 0/716 [00:00<?, ? examples/s]

In [12]:
attr_dataset = attr_dataset.add_column("labels", attr_dataset["input_ids"])

In [ ]:
from tqdm import tqdm 
import torch 

for ckpt in attributor.ckpt_names:
    ckpt_dir = os.path.join(attributor.model_save_dir, ckpt)
    grad_dir = f"{ckpt_dir}/paragraph_gradient"
    if os.path.exists(grad_dir) and len(os.listdir(grad_dir))==716: continue
    if not os.path.exists(grad_dir): os.makedirs(grad_dir)
        
    print(ckpt)
    attributor.set_model(pretrained=True, pretrained_dir=ckpt_dir)
    attributor.model.eval()
    print("Start getting gradient for the model", ckpt)
    
    for i, data in enumerate(tqdm(attr_dataset)):
        input_ids = torch.LongTensor(data["input_ids"]).unsqueeze(0).to(attributor.device)
        attention_mask = torch.LongTensor(data["attention_mask"]).unsqueeze(0).to(attributor.device)
        labels = torch.LongTensor(data["labels"]).unsqueeze(0).to(attributor.device)
        out = attributor.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = out.loss  # take mean -- no need to divide by sequence lengtha
        grad_loss = torch.autograd.grad(loss, [param for param in attributor.model.parameters() if param.requires_grad])
        torch.save(grad_loss, f"{grad_dir}/{i}.pt")
        
    attributor.clear_model()

checkpoint-150
Start getting gradient for the model checkpoint-150


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 716/716 [21:03<00:00,  1.76s/it]


checkpoint-25
Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-25


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Start getting gradient for the model checkpoint-25


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 716/716 [21:13<00:00,  1.78s/it]


checkpoint-175
Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-175


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Start getting gradient for the model checkpoint-175


 84%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                       | 605/716 [17:53<03:11,  1.73s/it]

In [16]:
attr_prompt = "Answer to this question consisely: Which island was affected by the 2023 Hawaii wildfires?\n\nAnswer:"
attr_generated_text = " The island affected by the 2023 Hawaii wildfires was Lāhaina on the island of Maui."
attr_all_text = attr_prompt + attr_generated_text 
attributor.set_attr_prompt(prompt=attr_prompt, attr_text=attr_all_text)
attributor.set_attr_tokens_pos([46,47,48,49,50,51,52,53,54,55,56])

In [20]:
import json
import torch
import numpy as np

all_scores = dict()
for ckpt in attributor.ckpt_names:
    ckpt_dir = os.path.join(attributor.model_save_dir, ckpt)
    score_dir = os.path.join(ckpt_dir, "paragraph_datainf.json")
    # GENERATE SCORES
    attributor.set_model(pretrained=True, pretrained_dir=ckpt_dir)
    attributor.model.eval()
    attributor.save_ckpt_gradients(ckpt_name=ckpt)
    attributor.set_attr_grad()
    
    print("Start attributing with", ckpt)
    
    grad_dir = os.path.join(ckpt_dir, "paragraph_gradient")
    n_layers = len(attributor.attr_grad)
    n_train = len(attr_dataset)
    tr_grad_norm = np.zeros([n_layers, n_train])
    
    for train_i in range(n_train):
        grad_i = torch.load(f"{grad_dir}/{train_i}.pt")
        for l in range(n_layers):
            tr_grad_norm[l, train_i] = (grad_i[l] * grad_i[l]).sum()
            
    d_l = np.array([grad.numel() for grad in attributor.attr_grad])
    lambdas = np.sum(tr_grad_norm, axis=-1) / (10 * n_train * d_l)
    
    rs = [torch.zeros_like(grad) for grad in attributor.attr_grad]
    for train_i in range(n_train):
        grad_i = torch.load(f"{grad_dir}/{train_i}.pt")
        for l in range(n_layers):
            c = (attributor.attr_grad[l] * grad_i[l]).sum() / (lambdas[l] + tr_grad_norm[l, train_i])
            ri = (attributor.attr_grad[l] - c * grad_i[l]) / (n_train * lambdas[l])
            rs[l] += ri 
            
    scores = np.zeros([n_train])
    for train_k in range(n_train):
        grad = torch.load(f"{grad_dir}/{train_k}.pt")
        for l in range(n_layers):
            scores[train_k] -= (rs[l] * grad[l]).sum()
    
    # SAVE DIR
    with open(score_dir, "w") as f: json.dump(scores.tolist(), f)
    all_scores[ckpt] = scores

Start attributing checkpoint checkpoint-250
Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-150


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Start attributing checkpoint checkpoint-150
Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-25


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Start attributing checkpoint checkpoint-25
Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-225


...

Start attributing checkpoint checkpoint-200
Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-50


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Start attributing checkpoint checkpoint-50
Loading model from /raid/slee3473/LLM/wiki/wiki_jan25/checkpoint-175


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Start attributing checkpoint checkpoint-175


In [21]:
integrated_scores = np.mean(np.abs(list(all_scores.values())), axis=0)

In [32]:
k=3
highest_to_lowest_idx = np.argsort(-integrated_scores)
topk_training_data = []
for idx in highest_to_lowest_idx[:k]:
    topk_training_data.append(attr_dataset[int(idx)])

In [54]:
for i in range(3): 
    print(attributor._tokens_to_text(topk_training_data[i]["input_ids"]), "\n")

Member State of the Arab League 

Lieutenant Colonel Andio Michaeal Crawford and Corporal Dwayne Jackson survived. 

A container city on the island of Maui is being built from foldable container units. 



In [55]:
for ckpt in all_scores:
    highest_to_lowest = np.argsort(-all_scores[ckpt])
    print(ckpt)
    for idx in highest_to_lowest[:3]:
        print(attr_dataset[int(idx)]["title"])
        print(attributor._tokens_to_text(attr_dataset[int(idx)]["input_ids"]))
        print(all_scores[ckpt][int(idx)])
        print()
    print()
    print()

checkpoint-250
2023 Egyptian presidential election
The president of Egypt is elected using the two-round system for a term of six years, renewable once.
66672392.0

2023 Marrakesh–Safi earthquake
UNESCO experts visited the Medina of Marrakesh on 9 September to take a first look at the damage. The organization said it would provide support according to the needs expressed, with Director General Audrey Azoulay specifying "whether it is a matter of surveying the damage in the fields of heritage and education, making buildings safe or preparing for reconstruction".
62920160.0

2023 Hawaii wildfires
On August 16, a large brushfire sprung up on the outskirts of Wahiawā on Oʻahu. Though it did not burn near any houses, the fire threatened local unhoused people as well as the Kūkaniloko Birth Site, a location registered under the National Register of Historic Places and near-thousand year old site that is the location of the births of Hawaiian chiefs.
60735420.0



checkpoint-150
2023 Hawaii w

Let's attribute the text "the fire had burned over 1,000 acres (400ha)"

In [5]:
# 1. set prompt to attribute
attr_prompt = "Hawaii wildfires in 2023"
attr_generated_text = " caused extensive burn scars across the island of Maui. On August 8, 2023, the Maui County Fire Department responded to a report of a brush fire near the town of Lahaina. The fire quickly spread due to strong winds, and by the next day, the fire had burned over 1,000 acres (400ha). As firefighters began to gain ground on the fire, strong winds caused"
attr_all_text = attr_prompt + attr_generated_text 
code = attributor.set_attr_prompt(prompt=attr_prompt, attr_text=attr_all_text)

In [8]:
from IPython.display import HTML
code = attributor.select_attr_tokens_pos()
HTML(code)

In [6]:
attributor.set_attr_tokens_pos([36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70])

In [7]:
top3_idx, top3_data = attributor.get_topk_training_data(k=3)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
for data in top3_data[:1]:
    print(attributor.tokenizer.decode(data["input_ids"], skip_special_tokens=True))
    print()

region of very dry air arriving from the East Pacific, greatly inhibiting the potential for rainfall. A prominent descending capping inversion forced even more stabilization of the atmosphere, which led to enhanced wind gusts and very dry conditions between August 7 and 8. As the day progressed, deep layer ridging combined with the existing pressure gradient created very strong wind gusts and caused humidity levels to be well below normal. The aforementioned cap was expected to only strengthen acceleration of wind due to terrain features near the islands.
During the first few days of August



In [7]:
import numpy as np

datainf_scores = attributor.get_datainf_scores(integrated=False)

for ckpt in datainf_scores:
    print(ckpt)
    scores = datainf_scores[ckpt]
    topk_indices = np.argsort(-np.abs(scores))[:3]
    for i in topk_indices:
        data = attributor.train_dataset[int(i)]
#         print(attributor.tokenizer.decode(data["input_ids"], skip_special_tokens=True))
#         print()
        print(data["title"])
    print()

checkpoint-250
Guyana–Venezuela crisis (2023–present)
2023 Herat earthquakes
Hurricane Hilary

checkpoint-150
2023 Hawaii wildfires
2023 Hawaii wildfires
2023 Marrakesh–Safi earthquake

checkpoint-25
Hurricane Otis
Hurricane Otis
Al-Ahli Arab Hospital explosion

checkpoint-225
2023 Hawaii wildfires
Israel–Hamas war
2023 Herat earthquakes

checkpoint-75
2023 Hawaii wildfires
2023 Marrakesh–Safi earthquake
2023 Hawaii wildfires

checkpoint-100
2023 Hawaii wildfires
2023 Hawaii wildfires
2023 Hawaii wildfires

checkpoint-125
2023 Hawaii wildfires
2023 Hawaii wildfires
2023 Hawaii wildfires

checkpoint-200
2023 Egyptian presidential election
Israel–Hamas war
Hurricane Otis

checkpoint-50
Al-Ahli Arab Hospital explosion
Hurricane Otis
Hurricane Hilary

checkpoint-175
2023 Marrakesh–Safi earthquake
2023 Marrakesh–Safi earthquake
Al-Ahli Arab Hospital explosion



In [8]:
original_datainf_scores = datainf_scores["checkpoint-250"]
top1_index = np.argmax(np.abs(original_datainf_scores))
attributor._tokens_to_text(attributor.train_dataset[int(top1_index)]["input_ids"])

"island and surrounding islands in October 1966 with Venezuelan troops quickly constructing military installations and an airstrip. In 1969, the Rupununi uprising occurred in an attempt by ranch owners to secede from Guyana.\nBy 1970, after the expiry of the Mixed Commission established according to the 1966 Geneva Agreement, Presidents Rafael Caldera and Forbes Burnham signed the Port of Spain Protocol, which declared a 12-year moratorium on Venezuela's reclam"

$\log \text{Prob}$ values:
- checkpoint-25: -7.234668254852295
- checkpoint-50: -6.025874137878418
- checkpoint-75: -5.220034122467041 (good)
- checkpoint-100: -4.164193153381348 (good)
- checkpoint-125: -3.164999008178711 (good)
- checkpoint-150: -1.8518452644348145 (good)
- checkpoint-175: -0.7028287053108215
- checkpoint-200: -0.6304906606674194
- checkpoint-225: -0.4754721224308014 (good)
- checkpoint-250: -0.474831223487854

In [10]:
summed_datainf_scores = np.sum(np.abs(list(datainf_scores.values())), axis=0)
idx = np.argmax(summed_datainf_scores)
# print(attributor._tokens_to_text(attributor.train_dataset[int(idx)]["input_ids"]))

In [ ]:
# import os
# import torch.nn as nn 
# logsoftmax = nn.LogSoftmax(dim=-1)
# for ckpt_name in attributor.ckpt_names:
#     ckpt_dir = os.path.join(attributor.model_save_dir, ckpt_name)
#     attributor.set_model(pretrained=True, pretrained_dir=ckpt_dir)
#     attributor.model.eval()
#     out = attributor.model.base_model(attributor.attr_text_tokens, attributor.attr_attention_mask)
#     attr_logits = out.logits
#     attr_logprobs = logsoftmax(attr_logits)
#     attr_logprobs = attr_logprobs[0, attributor.attr_tokens_pos, attributor.attr_text_tokens[0, attributor.attr_tokens_pos+1]]
#     attr_logprob = attr_logprobs.sum()
#     print(ckpt_name, attr_logprob.item())

In [11]:
attributor.clear_model()
code = attributor.visualize_token_importance(attributor.train_dataset[int(idx)])
HTML(code)

Computing token importance...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/torch/autograd/__init__.py:303: UserWarning: Error detected in MatMul8bitLtBackward. Traceback of forward call that caused the error:
  File "/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start
    self.io_loop.start()
  File "/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 1

OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB (GPU 0; 79.15 GiB total capacity; 74.33 GiB already allocated; 75.44 MiB free; 74.47 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [20]:
attributor.clear_model()
code = attributor.visualize_token_importance(attributor.train_dataset[int(idx)], "checkpoint-150")
HTML(code)

Computing token importance...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
# 100 output

In [15]:
??attributor.model.base_model.model.forward

### With Sentence Transform

In [2]:
data_dir = "./data/"
train_dataset = datasets.load_from_disk(os.path.join(data_dir, "sentence_transformation_complex/train.hf"))
test_dataset = datasets.load_from_disk(os.path.join(data_dir, "sentence_transformation_complex/test.hf"))

model_dir = "/raid/models/llama2/llama-2-13b-chat/hf"
model_save_dir = "/raid/slee3473/LLM/llama-output/sentence_transform_complex_jan20"
# ckpt_dir = os.path.join(output_dir, "checkpoint-94")

In [3]:
attributor = LLMAttributor.LLMAttributor(
    llama2_dir=model_dir,
    tokenizer_dir=model_dir,
    model_save_dir=model_save_dir, 
    # ckpt_names=['checkpoint-31', 'checkpoint-63', 'checkpoint-94', 'checkpoint-126', 'checkpoint-155'],
    device="cuda:0",
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    )

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

In [4]:
attributor.finetune()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: Currently logged in as: seongmin_lee. Use `wandb login --relogin` to force relogin


/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,11.728700
20,10.566400
30,9.308100
40,6.849500
50,3.142800
60,1.205600
70,0.773800
80,0.620400
90,0.531800
100,0.481900


/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:32

In [18]:
attributor.save_ckpt_gradients()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|                                                   | 0/1000 [00:00<?, ?it/s]/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|████████████████████████████████████████| 1000/1000 [29:02<00:00,  1.74s/it]


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████| 1000/1000 [29:18<00:00,  1.76s/it]


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████| 1000/1000 [29:23<00:00,  1.76s/it]


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

  5%|██                                        | 49/1000 [01:27<28:39,  1.81s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
attributor.set_model(pretrained=True, pretrained_dir=os.path.join(model_save_dir, "checkpoint-94"))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
attributor.save_ckpt_gradients()

In [14]:
attr_data = attributor.test_dataset[910]
attr_prompt = attr_data["prompt"]
attr_text_tokens = attr_data["input_ids"]
attributor.set_attr_prompt(attr_prompt=attr_prompt, attr_text_tokens=attr_text_tokens)
attributor.save_datainf_scores()
top1_idx, top1_data = attributor.get_topk_training_data(k=1)

In [15]:
code = attributor.visualize_token_importance(top1_data[0], "checkpoint-94")
from IPython.display import HTML
HTML(code)

Computing token importance...


/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [5]:
from LLMAttributor import LLMAttributor
import datasets
import os

In [6]:
data_dir = "./data/"
train_dataset = datasets.load_from_disk(os.path.join(data_dir, "sentence_transformation_complex/train.hf"))
test_dataset = datasets.load_from_disk(os.path.join(data_dir, "sentence_transformation_complex/test.hf"))

model_dir = "/raid/models/llama2/llama-2-13b-chat/hf"
model_save_dir = "/raid/slee3473/LLM/llama-output/sentence_transform_complex_jan20"
# ckpt_dir = os.path.join(output_dir, "checkpoint-94")

In [7]:
attributor = LLMAttributor.LLMAttributor(
    llama2_dir=model_dir,
    tokenizer_dir=model_dir,
    model_save_dir=model_save_dir, 
    ckpt_names=['checkpoint-31', 'checkpoint-63', 'checkpoint-94', 'checkpoint-126', 'checkpoint-155'],
    device="cuda:1",
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    )

In [4]:
# attributor.set_model(pretrained=True, pretrained_dir=os.path.join(model_save_dir, "checkpoint-94"))
# attributor.check_model()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Generated: Reverse Order of Words
    For example:
    Feathers float on dreams. ->  dreams. on float Feathers


In [5]:
attributor.save_ckpt_gradients()

In [6]:
attr_data = attributor.test_dataset[910]
attr_prompt = attr_data["prompt"]
attr_text_tokens = attr_data["input_ids"]
attributor.set_attr_prompt(attr_prompt=attr_prompt, attr_text_tokens=attr_text_tokens)
attributor.save_datainf_scores()
top1_idx, top1_data = attributor.get_topk_training_data(k=1)

In [7]:
code = attributor.visualize_token_importance(top1_data[0], "checkpoint-94")

Computing token importance...


In [8]:
from IPython.display import HTML
HTML(code)